# DistilBERT Endian Classifier Evaluation

## Prerequisites

In [ ]:
!pip install transformers[torch] -U
!pip install accelerate -U
!pip install datasets
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 49.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.3 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.0 MB

## Load model and tokenizer

In [ ]:
# Load model andf tokenizer

from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = "ryfye181/distilbert_endian_classifier"

model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/732 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

## Load Dataset

In [ ]:
# Load test dataset

from datasets import load_dataset
dataset = load_dataset("ryfye181/endianness", data_dir="test", split='train')

dataset = dataset.shuffle(seed=42)

## Evaluate

In [ ]:
import evaluate
import torch
from tqdm import tqdm

def compute_accuracy(model, tokenizer, dataset, device):
    # Initialize accuracy metric
    accuracy = evaluate.load("accuracy")
    label2id = {"little": 0, "big": 1}

    # Configure model for evaluation
    model.eval()
    model.to(device)

    # Initialize lists to store references and predictions
    all_references = []
    all_predictions = []

    # Convert dataset to a list of dictionaries for faster iteration
    dataset_list = dataset.to_pandas().to_dict('records')

     # Set batch size for processing
    batch_size = 32

    # Create a tqdm progress bar for accuracy
    progress_bar = tqdm(range(0, len(dataset_list), batch_size), desc="Processing")

    # Process the dataset in batches
    for i in progress_bar:
        batch = dataset_list[i:i+batch_size]

        # Tokenize the batch
        inputs = tokenizer([row['data'] for row in batch], return_tensors="pt", padding=True, truncation=True).to(device)

        # Perform inference
        with torch.no_grad():
            logits = model(**inputs).logits
            predicted_class_ids = logits.argmax(dim=-1).tolist()

        # Extend the references and predictions lists
        batch_references = [label2id[row['endianness']] for row in batch]
        all_references.extend(batch_references)
        all_predictions.extend(predicted_class_ids)

        # Calculate and display current accuracy
        current_accuracy = accuracy.compute(references=all_references, predictions=all_predictions)

        # Update the progress bar description with current accuracy
        progress_bar.set_description(f"Processing (Current Accuracy: {current_accuracy['accuracy']:.4f})")

    # Compute and return the final accuracy
    final_accuracy = accuracy.compute(references=all_references, predictions=all_predictions)
    print(f"\nFinal Accuracy: {final_accuracy['accuracy']:.4f}")
    return final_accuracy

# Usage
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
result = compute_accuracy(model, tokenizer, dataset, device)

Using device: cuda


Processing (Current Accuracy: 0.7725):   4%|▍         | 1394/32789 [09:50<3:41:42,  2.36it/s]


KeyboardInterrupt: 